# KOSTRA skjema J og L

## Henter nødvendige pakker
**For løpet Kostra0J_0L trengs følgende pakker:**  
   - Pandas
   - Numpy
   - Glob (for import fra Linux - ikke nødvendig når hele flyten er over)
   - Selvlagde funksjoner lagret i fil funksjoner.py

In [ ]:
import pandas as pd
import numpy as np
import glob
exec(open('../funksjoner.py').read())

## Angi årgang som produseres

In [ ]:
aar=input('Angi årgang for filene:' )
aar_1t = str(int(aar) - 1)
aar_t1 = str(int(aar) + 1)
print("Du lager nå filer for " + aar)

## Henter nødvendige datafiler
**For løpet Kostra0J_0L trengs følgende datafiler:**
- Import fra Driller
    - Skjema KOSTRA0J
    - Skjema KOSTRA0L
- Lister fra KLASS
    - Kodeliste for Kostra kapitler
- Regnskapsfiler
    - Kostra0b_0d_levert gjeldene årgang

In [ ]:
#Henter nødvendige filer
kostra0j = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/import_driller/" + aar + "/kostra0j.parquet")
kostra0l = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/import_driller/" + aar + "/kostra0l.parquet")
kap = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kapittel.parquet").astype(str)
levert = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/kostra0b_0d_levert_" + aar + ".parquet")
eierandel = pd.read_csv("/ssb/bruker/mtk/Testdata/inndata/Eierandel_" + aar + "_KOMPIS.csv", sep=';', encoding='latin-1')
kassa = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/kostra0b_0d_basis_" + aar + ".parquet")

## Bearbeider regnskapsfilene

### Slår sammen Kostra0I og Kostra0K

In [ ]:
jl = pd.concat([kostra0j, kostra0l])

### Ordner data
- Definerer rett datatype
- Endrer kolonnenavn
    - Fra store til små bokstaver
    - Endrer navn
- Fjerner manglende observasjoner
    - Har ikke sjekket om denne funksjonen faktisk fungerer da datasettet mitt ikke har noen manglende observasjonen. Jeg fjerne na. Men det kan jo være manglende registreres som NaN? Dette må kontrolleres!

In [ ]:
#Endrer datatype.
jl = jl.astype({'ART_SEKTOR':str,'FUNKSJON_KAPITTEL':str,'KONTOKLASSE':str,'PERIODE':str,'ORG_NR':str})

In [ ]:
#Endrer headingen fra store til små bokstaver og navn på to av variablene
jl.columns = [x.lower() for x in jl.columns]
jl.rename(columns={'funksjon_kapittel':'kapittel', 'org_nr':'saerbedrift'}, inplace = True)

In [ ]:
jl = jl.groupby(['kapittel', 'kontoklasse', 'periode', 'saerbedrift'])[['belop']].sum().reset_index()

In [ ]:
#Filtrerer bort alle rader med manglede verdier. Fungerer det? Må testes på et annet datasett som faktisk har manglende verdier... Denne er ikke med i KOMPIS?
jl_null = jl.dropna()

In [ ]:
kontroll_merge(jl,jl_null)

### Snur fortegn for inntektsarter
- Endrer alle verdier som har art høyere eller lik 600 til negative tall
- Koden under snur fortegn på alle verdier hvor art starter på siffer 6 eller mer.

In [ ]:
jl_snudd = jl_null.copy()

In [ ]:
jl_snudd['belop']=np.where((jl_snudd.kapittel >= '31') & (jl_snudd.kapittel <= '5990'),  jl_snudd.belop*-1, jl_snudd.belop)

## Filtrerer gyldige kapitler
**Kapittel listen**
- Hentes direkte fra KLASS via API i egen notebook (API Klass). 
- Filen laget i API-notebooken er lastet inn helt i starten av denne boken.

**Kontroll**
- Funksjonen kontroll_merge er laget i egen notebook (funksjoner.py)
- Kontrollen lister ut alle rader som er fjernet i mergen over. Dersom ingen rader listes ut ble ingenting fjernet i operasjonen over.

### Kapittelfilter
Merger alle gylige kapitler med regnskapsfilen. Dette fjerner alle reder med ugyldige kapitler

In [ ]:
kap=kap.drop(columns={'periode'})
gyldige_kap = list(kap.columns.values)
k1 = jl_snudd.set_index(gyldige_kap).index
k2 = kap.set_index(gyldige_kap).index
kapittelfilter = jl_snudd[k1.isin(k2)]

#### Kontrollerer fjernede rader

In [ ]:
kontroll_merge(jl_snudd, kapittelfilter)

# * Føreløpig fil - kontrollert *

In [ ]:
kostra0j_0l_kontrollert = kapittelfilter.copy()

## Eierandeler

- Denne filen skal lages direkte i Python. Foreløpig hentes den fra linux. 
- Siden filen hentes fra Linux må ledene nuller legges til

In [ ]:
eierandel['region']=eierandel['region'].apply(lambda x: '{0:0>4}'.format(x))
eierandel = eierandel.astype({'periode':str,'saerbedrift':str})

Slår sammen eierandelene med listen over kommuner som har levert. På denne måten filtreres kommuner som ikke har levert bort.

In [ ]:
eier_levert = pd.merge(eierandel, levert, on=['region'])
eier_levert = eier_levert.astype({'periode':str,'saerbedrift':str})

#### Kontrollerer fjernede rader

In [ ]:
kontroll_merge(eierandel, eier_levert)

### Fordeler særbedriftenes regnskap

In [ ]:
fordelt = pd.merge(kostra0j_0l_kontrollert[['kapittel','kontoklasse','belop','saerbedrift']],
                   eier_levert[['periode','saerbedrift','region','andel']], left_on=['saerbedrift'], right_on=['saerbedrift'])

Fordeler særbedriftenes regnskap på kommunene ved å gange beløp og andel

In [ ]:
fordelt['fordelt'] = fordelt['belop'] * fordelt['andel']

Fjerner overflødige kolonner

In [ ]:
fordelt = fordelt.drop(columns={'andel','belop'})
fordelt = fordelt.rename(columns={'fordelt':'belop'})

## Lager Oslo Fylkeskommune
Tallene fra Oslo skal være med både for kommune og fylkeskommune. Vi må derfor kopiere alle Oslo tall og endre region på de kopierte radene fra 0301 til 0300. 

In [ ]:
#Henter ut Oslo og endrer regionskode til 0300. Legger Oslo fylkeskommune på arbeidsfilen. 
oslo=fordelt[(fordelt['region']=='0301')].copy()
oslo['region'] = oslo['region'].str.replace('0301', '0300')

Slår sammen Oslo Fylkeskommune med regnskapsfilen

In [ ]:
kostra0j_0l_fordelt = pd.concat([fordelt, oslo])

## Fjerner organisasjonsnummer og summerer langs regnskapet

In [ ]:
kostra0j_0l_fordelt = kostra0j_0l_fordelt.groupby(['kapittel','kontoklasse','periode','region']).sum().reset_index()

In [ ]:
kostra0j_0l_fordelt['regnskapsomfang'] = 'sbedr'

## Samler kasse og særbedrift

In [ ]:
balanse_basis = pd.concat([kassa, kostra0j_0l_fordelt])

# Ferdig fil
- Lagrer fil som er klar som input til KostraRegnskaps-metode

In [ ]:
balanse_basis.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/balanse_basis_" + aar + ".parquet")

#### Kontrollfilter

In [ ]:
#sjekk=kostra0j_0l_fordelt[(kostra0j_0l_fordelt['region']=='4601') & (kostra0j_0l_fordelt['art']=='510') ].copy()
#sjekk

In [ ]:
#agg = sjekk.groupby(['art','periode','kontoklasse','region'])[['belop']].sum().reset_index()
#agg

In [ ]:
#kostra0j_0l_fordelt

In [ ]:
balanse_basis